## 3. WELCOME TO LANGCHAIN

In [1]:
from langchain_openai import ChatOpenAI # type: ignore
chat = ChatOpenAI(model_name="o4-mini")
question="How many planets are there?"
# chat_response = chat.predict(question) #this method(predict) will soon be deprecated
chat_response = chat.invoke(question)
print(chat_response)


/Users/boyoon/Documents/GitHub/fullstack-gpt-2025/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/var/folders/n3/6fbssxjx1x7bxcnvp_nn7g3r0000gn/T/ipykernel_25744/292855851.py:4: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chat_response = chat.predict(question)


If you mean “planets in our Solar System,” there are eight, in order from the Sun:  
1. Mercury  
2. Venus  
3. Earth  
4. Mars  
5. Jupiter  
6. Saturn  
7. Uranus  
8. Neptune  

(Pluto and a handful of other bodies—Ceres, Eris, Haumea and Makemake—are classified as “dwarf planets” by the International Astronomical Union.)

If you widen the scope to the rest of the galaxy (and beyond), astronomers have now confirmed over 5 000 exoplanets orbiting other stars—and ongoing surveys suggest there may be literally billions more. The total number of planets in the universe is unknown (and almost certainly vast).


### 3.1 Predict Messages

In [7]:
from langchain_openai import ChatOpenAI # type: ignore
from langchain.schema import HumanMessage, AIMessage, SystemMessage # type: ignore

chat = ChatOpenAI(model_name="o4-mini")

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in Italin."),
    AIMessage(content="Ciao! Mi chiamo Paolo!"),
    HumanMessage(content="What is the distance between Mexico and France? And what's your name?")
]

# res = chat.predict_messages(messages) #this method(predict_messages) will be soon deprecated.
res = chat.invoke(messages)
print(res)

content='La distanza in linea d’aria tra Città del Messico e Parigi (capitali di Messico e Francia) è di circa 9.100 chilometri.  \nMi chiamo Paolo.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 445, 'prompt_tokens': 52, 'total_tokens': 497, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'o4-mini-2025-04-16', 'system_fingerprint': None, 'id': 'chatcmpl-BOpSZ5ykJifXMVpgHmW83n3XXMp1S', 'finish_reason': 'stop', 'logprobs': None} id='run-0879f96f-2566-4069-a11b-f08a2aff606d-0' usage_metadata={'input_tokens': 52, 'output_tokens': 445, 'total_tokens': 497, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 384}}


### 3.2 Prompt Templates
* template을 디스크에 저장하고 load할 수 있음.

In [10]:
from langchain_openai import ChatOpenAI # type: ignore
from langchain.schema import HumanMessage, AIMessage, SystemMessage # type: ignore
from langchain.prompts import PromptTemplate, ChatPromptTemplate # type: ignore

chat = ChatOpenAI(model_name="o4-mini")

#PromptTempalte
template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}?")
prompt = template.format(country_a="Korea", country_b="France")
chat.invoke(prompt)

#ChatPromptTemplate
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao! Mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}? And what's your name?")
])

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand"
)

chat.invoke(prompt)

AIMessage(content='Η ευθύγραμμη (αεροπορική) απόσταση μεταξύ Πόλης του Μεξικού και Μπανγκόκ στην Ταϊλάνδη είναι περίπου 15\xa0900 χιλιόμετρα.  \nΤο όνομά μου είναι Βοηθός.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1040, 'prompt_tokens': 52, 'total_tokens': 1092, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 960, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'o4-mini-2025-04-16', 'system_fingerprint': None, 'id': 'chatcmpl-BOpeZkfHyYLZMyPpY2cKQTLEGLPBO', 'finish_reason': 'stop', 'logprobs': None}, id='run-8f1c67e9-7445-4c06-8b18-226395c20f59-0', usage_metadata={'input_tokens': 52, 'output_tokens': 1040, 'total_tokens': 1092, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 960}})

### 3.3 OutputParser and LCEL
* LCEL : LangChain Expression Language

In [22]:
from langchain_openai import ChatOpenAI # type: ignore
from langchain.schema import BaseOutputParser #type: ignore
from langchain.prompts import ChatPromptTemplate # type: ignore

chat = ChatOpenAI(model_name="o4-mini")

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommaOutputParser()

template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma-seperated list of {max_items} in lowercase. DO NOT reply with anything else."),
    ("human", "{question}")
])


prompt = template.format_messages(
    max_items = 10,
    question="Give me 10 most popular colors."
)

colors = chat.invoke(prompt)
p.parse(colors.content)

['blue',
 'red',
 'green',
 'black',
 'white',
 'yellow',
 'purple',
 'orange',
 'pink',
 'brown']

In [23]:
# chain을 이용해서 위 코드를 개선할 수 있음
from langchain_openai import ChatOpenAI # type: ignore
from langchain.schema import BaseOutputParser #type: ignore
from langchain.prompts import ChatPromptTemplate # type: ignore

chat = ChatOpenAI(model_name="o4-mini")

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommaOutputParser()

template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma-seperated list of {max_items} in lowercase. DO NOT reply with anything else."),
    ("human", "{question}")
])

chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items":5, 
    "question":"Give me the most popular pokemons."
})

['pikachu', 'charizard', 'eevee', 'mewtwo', 'bulbasaur']

### 3.4 Chaining Chains
* callbacks & streaming=True : 답변 생성이 전부 완료될 때까지 기다리지 않음. 생성되는 즉시 결과를 확인.
* callback: LLM에서 일어나는 다양한 event를 감지할 수 있음.

In [25]:
from langchain_openai import ChatOpenAI # type: ignore
from langchain.prompts import ChatPromptTemplate # type: ignore
from langchain.callbacks import StreamingStdOutCallbackHandler # type: ignore

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
    )

#normal recipes
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy-to-follow recipes for any type of cuising with easy-to-find ingredients."),
    ("human", "I want to cook {country} food.")
])
chef_chain = chef_prompt | chat

#vegetarian recipes
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized in making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative, just say you don't know how to replace it."),
    ("human", "{recipe}")
])
veg_chef_chain = veg_chef_prompt | chat

#mixing two chains
final_chain = {"recipe":chef_chain} | veg_chef_chain

#calling OpenAI API
final_chain.invoke({
    "country":"korean"
})

Here are three classic, easy-to-find Korean dishes you can try at home. Pick one (or all three!), gather the ingredients, and follow the step‑by‑step.  

1. Kimchi Fried Rice (Kimchi Bokkeumbap)  
Ingredients (2 servings)  
 • 2 cups cold cooked rice (preferably a day old)  
 • 1 cup well-fermented kimchi, chopped (reserve 2 Tbsp kimchi juice)  
 • ½ onion, diced  
 • 1 small carrot, julienned (optional)  
 • 2 scallions, sliced  
 • 1 Tbsp gochujang (Korean red‑pepper paste) or 1 tsp gochugaru + 1 Tbsp soy sauce  
 • 1 Tbsp soy sauce  
 • 1 tsp sugar  
 • 1 tsp sesame oil  
 • 1 Tbsp neutral oil (vegetable/canola)  
 • 2 eggs  
 • Toasted sesame seeds (for garnish)  

Steps  
1. Prep: Heat a large nonstick skillet over medium–high.  
2. Aromatics: Add neutral oil → sauté onion and carrot 2–3 min until soft.  
3. Kimchi: Add chopped kimchi + 2 Tbsp kimchi juice → cook 2 min.  
4. Seasoning: Stir in gochujang (or gochugaru+soy), soy sauce, sugar → mix.  
5. Rice: Break up cold rice with

AIMessage(content='Here’s how you can turn each of these into a vegetarian (or even vegan) dish, with minimal tweaks to the original recipes.  \n\n1. Kimchi Fried Rice  \n   • Kimchi: Most store‑bought kimchis use fish sauce or shrimp paste. Look for “vegan kimchi” at Korean markets or make your own by substituting the fish ingredients with a little crushed nori (seaweed) and a splash of soy sauce or tamari in your chili paste.  \n   • Eggs (optional): If you eat eggs, keep the sunny‑side‑up as written. For a vegan version, crumble firm tofu and sauté it in a little oil, a pinch of turmeric for color, plus salt and black salt (kala namak) for that eggy aroma. Stir it in at the very end or serve on top.  \n\n2. “Mushroom Bulgogi” (in place of Beef Bulgogi)  \n   Ingredients (2–3 servings)  \n    • 300\xa0g (10\xa0oz) king oyster mushrooms (or shiitakes + cremini), thinly sliced lengthwise into “scallops”  \n    • 3\xa0Tbsp soy sauce  \n    • 2\xa0Tbsp brown sugar or maple syrup  \n    •

## 4. MODEL IO
* <b>Modules</b>: Model I/O, Retrieval, Chains, Agents, Memory, Callbacks
* <b>Model I/O</b> : Interface with language models<br/>
   - prompts, language models, output parsers
* <b>Retrieval</b>: 외부 데이터로 접근하여 이를 모델에 제공
   - document loaders, transformers, text embedding, vector stores, retrievers
* <b>Agents</b>: AI가 독립적으로 작동할 수 있게 함.
* <b>Callbacks</b>: Log and stream intermediate steps

### 4.1 FewShotPromptTemplate
* 모델에게 예제를 주는 것과 같음 (대답 포맷 등). 답변의 형식을 구구절절 설명하는 것보다 더 빠르고 정확할 수 있음.

In [27]:
#PromptTemplate을 이렇게 만들 수도 있음 (그냥 알고 있으라고)
t = PromptTemplate(
    template = "What is the capital of {country}?",
    input_variables = ["country"]
)
#same as PromptTemplate.from_template("What is the capital of {country}?")
t.format(country="France")

'What is the capital of France?'

In [35]:
from langchain_openai import ChatOpenAI # type: ignore
from langchain.prompts.few_shot import FewShotPromptTemplate # type: ignore
from langchain.prompts import PromptTemplate # type: ignore
from langchain.callbacks import StreamingStdOutCallbackHandler # type: ignore

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
    )

examples = [
    {"question":"What do you know about France?", 
    "answer":"capital is Paris, \nlanguage is French, \nfood is Bœuf bourguignon and onion soup."},
    {"question":"What do you know about Korea?", 
    "answer":"capital is Seoul, \nlanguage is Korean, \nfood is Kimchi and Bibimbap."},
    {"question":"What do you know about Italy?", 
    "answer":"capital is Rome, \nlanguage is Italian, \nfood is pasta and pizza."},
    {"question":"What do you know about Japan?", 
    "answer":"capital is Tokyo, \nlanguage is Japanese, \nfood is sushi and sashimi."}
]

example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

chain = prompt | chat
chain.invoke({"country":"Poland"})

AI: The capital is Warsaw, the language is Polish, and the food includes pierogi, kielbasa, and bigos.

AIMessage(content='AI: The capital is Warsaw, the language is Polish, and the food includes pierogi, kielbasa, and bigos.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-400ce6e5-c9b3-44c7-97c8-f8ae7dbe8190-0')

### 4.2 FewShotChatMessagePromptTemplate

In [36]:
from langchain_openai import ChatOpenAI # type: ignore
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate # type: ignore
from langchain.prompts import ChatPromptTemplate # type: ignore
from langchain.callbacks import StreamingStdOutCallbackHandler # type: ignore

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
    )

examples = [
    {"country":"France", 
    "answer":"capital is Paris, \nlanguage is French, \nfood is Bœuf bourguignon and onion soup."},
    {"country":"Korea", 
    "answer":"capital is Seoul, \nlanguage is Korean, \nfood is Kimchi and Bibimbap."},
    {"country":"Italy", 
    "answer":"capital is Rome, \nlanguage is Italian, \nfood is pasta and pizza."},
    {"country":"Japan", 
    "answer":"capital is Tokyo, \nlanguage is Japanese, \nfood is sushi and sashimi."}
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai","{answer}")
])

fewshot_example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a geography expert"),
    fewshot_example_prompt, #이전의 모든 example들
    ("human", "What do you know about {country}?")
])

chain = final_prompt | chat
chain.invoke("USA")


capital is Washington D.C., 
language is English, 
food is burgers and apple pie.

AIMessage(content='capital is Washington D.C., \nlanguage is English, \nfood is burgers and apple pie.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-343e758b-b342-4748-9a4a-df05eab6528d-0')

### 4.3 LengthBasedExampleSelector
* dynamic하게 example들을 선택 -> 셀 수 없을 정도로 많은 example들이 있을 수 있음. 전부 모델한테 줄 수는 없어.


In [42]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

from langchain_openai import ChatOpenAI # type: ignore
from langchain.prompts.few_shot import FewShotPromptTemplate # type: ignore
from langchain.prompts import PromptTemplate # type: ignore
from langchain.callbacks import StreamingStdOutCallbackHandler # type: ignore

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
    )

examples = [
    {"question":"What do you know about France?", 
    "answer":"capital is Paris, \nlanguage is French, \nfood is Bœuf bourguignon and onion soup."},
    {"question":"What do you know about Korea?", 
    "answer":"capital is Seoul, \nlanguage is Korean, \nfood is Kimchi and Bibimbap."},
    {"question":"What do you know about Italy?", 
    "answer":"capital is Rome, \nlanguage is Italian, \nfood is pasta and pizza."},
    {"question":"What do you know about Japan?", 
    "answer":"capital is Tokyo, \nlanguage is Japanese, \nfood is sushi and sashimi."}
]

example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

example_selector = LengthBasedExampleSelector(
    examples = examples,
    example_prompt = example_prompt,
    max_length=30
)

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector = example_selector, #전에는 그냥 모든 example들을 넣었음
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Netherlands")

'Human:What do you know about France?\nAI:capital is Paris, \nlanguage is French, \nfood is Bœuf bourguignon and onion soup.\n\nHuman: What do you know about Netherlands?'

In [46]:
# 랜덤하게 example들을 선택
from langchain.prompts.example_selector.base import BaseExampleSelector

from langchain_openai import ChatOpenAI # type: ignore
from langchain.prompts.few_shot import FewShotPromptTemplate # type: ignore
from langchain.prompts import PromptTemplate # type: ignore
from langchain.callbacks import StreamingStdOutCallbackHandler # type: ignore

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
    )

examples = [
    {"question":"What do you know about France?", 
    "answer":"capital is Paris, \nlanguage is French, \nfood is Bœuf bourguignon and onion soup."},
    {"question":"What do you know about Korea?", 
    "answer":"capital is Seoul, \nlanguage is Korean, \nfood is Kimchi and Bibimbap."},
    {"question":"What do you know about Italy?", 
    "answer":"capital is Rome, \nlanguage is Italian, \nfood is pasta and pizza."},
    {"question":"What do you know about Japan?", 
    "answer":"capital is Tokyo, \nlanguage is Japanese, \nfood is sushi and sashimi."}
]

# 나만의 example selector를 만들 수 있다는 간단한 예시
class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example): #ExampleSelector 내부에 반드시 있어야 함 (없으면 에러)
        return super().add_example(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)] #하나의 example만 선택해서 리턴

example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples = examples
)

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector = example_selector, #전에는 그냥 모든 example들을 넣었음
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Netherlands")

'Human:What do you know about Italy?\nAI:capital is Rome, \nlanguage is Italian, \nfood is pasta and pizza.\n\nHuman: What do you know about Netherlands?'

### 4.4 Serialization and Composition
* serialize(직렬화): 불러오기, 저장 등
* compose: 작은 prompt templates들을 결합


In [50]:
# prompt 불러오기
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.json")
print(prompt.format(country="China"))

prompt = load_prompt("./prompt.yaml")
print(prompt.format(country="China"))

What is the capital of China?
What is the capital of China?


In [51]:
# 여러 개의 prompt를 하나로 합치기
from langchain.prompts.pipeline import PipelinePromptTemplate

from langchain_openai import ChatOpenAI # type: ignore
from langchain.callbacks import StreamingStdOutCallbackHandler # type: ignore
from langchain.prompts import PromptTemplate # type: ignore

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}.
"""
)
example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human:{example_question}
    You: {example_answer}
"""
)
start=PromptTemplate.from_template(
    """
    Start now!

    Human:{question}
    You:
"""
)
final = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
"""
)

prompts=[
    ("intro",intro),
    ("example",example),
    ("start",start)
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts
)

chain = full_prompt | chat
chain.invoke(
    {
        "character":"Pirate",
        "example_question":"What is your location?",
        "example_answer":"Arrrrg! That is a scret!! Arg arrrg!",
        "question": "What is your favorite food?"
    }
)

/var/folders/n3/6fbssxjx1x7bxcnvp_nn7g3r0000gn/T/ipykernel_25744/1575831411.py:52: LangChainDeprecationWarning: This class is deprecated. Please see the docstring below or at the link for a replacement option: https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.pipeline.PipelinePromptTemplate.html
  full_prompt = PipelinePromptTemplate(


Arrr matey! Me favorite food be grilled fish and salted pork, aye! Nothing like a good meal to keep me strength up on the high seas. Arrrr!

AIMessage(content='Arrr matey! Me favorite food be grilled fish and salted pork, aye! Nothing like a good meal to keep me strength up on the high seas. Arrrr!', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-8c1d7fe1-eb46-44e4-911a-06350539baab-0')

### 4.5 Caching
* 질문, 답변을 저장. 중복되는 API 콜을 줄일 수 있음.

In [54]:
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

from langchain_openai import ChatOpenAI # type: ignore

# set_llm_cache(InMemoryCache()) #이제부터 모든 response가 메모리에 저장됨. 컴퓨터를 재시동하면 없어짐.
set_llm_cache(SQLiteCache("cache.db")) #sqlite 데이터베이스 생성. 여기에 response를 저장함.
set_debug(True) #로그를 보여줌. 나중에 체인 작업을 할 때 매우 유용

chat=ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.1
)

chat.invoke("Give me a recipe of 김치볶음밥")

AIMessage(content='Ingredients:\n- 2 cups cooked rice\n- 1 cup kimchi, chopped\n- 1/2 cup kimchi juice\n- 1/2 cup diced pork belly or bacon\n- 1/2 onion, diced\n- 2 cloves garlic, minced\n- 1 tbsp gochujang (Korean red chili paste)\n- 1 tbsp soy sauce\n- 1 tbsp sesame oil\n- 1 green onion, chopped\n- 1 fried egg (optional)\n- Sesame seeds for garnish\n\nInstructions:\n1. Heat a large skillet over medium heat and add the diced pork belly or bacon. Cook until browned and crispy.\n2. Add the diced onion and minced garlic to the skillet and cook until softened.\n3. Add the chopped kimchi and cook for a few minutes until heated through.\n4. Stir in the gochujang, soy sauce, and kimchi juice. Mix well.\n5. Add the cooked rice to the skillet and stir to combine with the kimchi mixture.\n6. Drizzle with sesame oil and continue to cook for a few more minutes, stirring occasionally.\n7. Remove from heat and garnish with chopped green onion and sesame seeds.\n8. Serve hot with a fried egg on top,

In [55]:
chat.invoke("Give me a recipe of 김치볶음밥") #똑같은 질문을 두 번째 함 -> 답변하는데 0초 소요. 메모리에서 들고 왔으니까.

AIMessage(content='Ingredients:\n- 2 cups cooked rice\n- 1 cup kimchi, chopped\n- 1/2 cup kimchi juice\n- 1/2 cup diced pork belly or bacon\n- 1/2 onion, diced\n- 2 cloves garlic, minced\n- 1 tbsp gochujang (Korean red chili paste)\n- 1 tbsp soy sauce\n- 1 tbsp sesame oil\n- 1 green onion, chopped\n- 1 fried egg (optional)\n- Sesame seeds for garnish\n\nInstructions:\n1. Heat a large skillet over medium heat and add the diced pork belly or bacon. Cook until browned and crispy.\n2. Add the diced onion and minced garlic to the skillet and cook until softened.\n3. Add the chopped kimchi and cook for a few minutes until heated through.\n4. Stir in the gochujang, soy sauce, and kimchi juice. Mix well.\n5. Add the cooked rice to the skillet and stir to combine with the kimchi mixture.\n6. Drizzle with sesame oil and continue to cook for a few more minutes, stirring occasionally.\n7. Remove from heat and garnish with chopped green onion and sesame seeds.\n8. Serve hot with a fried egg on top,

### 4.6 Serialization
* 모델을 저장 & 불러오기

In [56]:
# 한 번 API 콜을 할 떄 소요되는 비용 체크
from langchain_openai import ChatOpenAI # type: ignore
from langchain.callbacks import get_openai_callback # type: ignore

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    temperature=0.1,
)

with get_openai_callback() as usage:
    kimchi_recipe = chat.invoke("Tell me how I can make Kimchi(겉절이)")
    print(usage)
    print(usage.total_cost) #가격에 대해서만 알고 싶다면
    print(usage.prompt_tokens) #내가 모델에 건네준 토큰
    print(usage.completion_tokens) #모델이 생성한 토큰

print(kimchi_recipe)


Tokens Used: 430
	Prompt Tokens: 22
		Prompt Tokens Cached: 0
	Completion Tokens: 408
		Reasoning Tokens: 0
Successful Requests: 1
Total Cost (USD): $0.0006230000000000001
content='Kimchi is a traditional Korean side dish made of fermented vegetables, most commonly napa cabbage and Korean radish. Here is a simple recipe to make kimchi at home:\n\nIngredients:\n- 1 napa cabbage\n- 1 Korean radish\n- 1/2 cup of coarse sea salt\n- 1/4 cup of Korean red pepper flakes (gochugaru)\n- 1 tablespoon of minced garlic\n- 1 tablespoon of minced ginger\n- 1 tablespoon of fish sauce\n- 1 tablespoon of sugar\n- 2 green onions, chopped\n- 1 carrot, julienned\n\nInstructions:\n1. Cut the napa cabbage in half lengthwise, then cut each half into quarters. Remove the core and chop the cabbage into bite-sized pieces. Peel the Korean radish and cut it into thin matchsticks.\n\n2. In a large bowl, dissolve the sea salt in water. Add the cabbage and radish, making sure they are fully submerged in the brine. L

In [65]:
#모델 저장하기
from langchain_openai import ChatOpenAI # type: ignore
chat = ChatOpenAI(
    model="gpt-3.5-turbo",
    max_completion_tokens=500, #말 많은거 싫어
    temperature=0.1 #너무 창의적인거 싫어
)
# chat.save("model.json") #model.json 파일이 생성됨 -> 더 이상 지원하지 않음. joblib 혹은 pickle을 사용해야 함.

# #모델 불러오기
from langchain.llms.loading import load_llm
# chat = load_llm("model.json")

## 5. MEMORY
* OpenAI의 Chat은 기본적으로 memory를 제공하지 않음 (stateless)
* 랭체인을 이용해서 implement

### 5.0 ConversationBufferMemory
* <b>Conversation Buffer Memory</b>: 대화 전체를 저장. prompt가 길어짐 -> 비싸!

In [67]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)#return_messages=False를 주면 메시지 객체가 아니라 문자열을 반환

memory.save_context({"input":"Hi!"}, {"output":"How are you?"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='How are you?', additional_kwargs={}, response_metadata={})]}

### 5.1 ConversationBufferWindowMemory
* <b>Conversation Buffer Window Memory</b>: 최근 n개의 대화를 저장 -> 최근 대화만 기억할 것

In [70]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})

add_message("1","1") #이 부분은 버려짐
add_message("2","2")
add_message("3","3")
add_message("4","4")
add_message("5","5")

memory.load_memory_variables({})



{'history': [HumanMessage(content='2', additional_kwargs={}, response_metadata={}),
  AIMessage(content='2', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='3', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='4', additional_kwargs={}, response_metadata={}),
  AIMessage(content='4', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='5', additional_kwargs={}, response_metadata={}),
  AIMessage(content='5', additional_kwargs={}, response_metadata={})]}

### 5.2 ConversationSummaryMemory
* <b>Conversation Summary Memory</b>: llm을 이용. 대화가 길어질수록 유용해짐.

In [74]:
from langchain.memory import ConversationSummaryMemory
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi! I'm Boyon, I live in Paris.", "Wow that is so cool!")
add_message("Paris is very beautiful.", "I wish I could go")
get_history() #진행된 대화가 별로 없음 -> 초반엔 대화보다 summary가 더 긺.

{'history': 'The human introduces themselves as Boyon from Paris. The AI responds with enthusiasm at the information shared, expressing a desire to visit the beautiful city of Paris.'}

### 5.3 ConversationSummaryBufferMemory
* <b>Conversation Summary Buffer Memory</b>: max_token_limit에 달하면, 최근 메시지를 제외한 메시지들을 summarize

In [75]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=20, #최근 max_token_limit개 토큰의 interactions를 제외하고 summarize
    return_messages=True
)

def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi! I'm Boyon, I live in Paris.", "Wow that is so cool!")
add_message("Paris is very beautiful.", "I wish I could go")
get_history() #진행된 대화가 별로 없음 -> 초반엔 대화보다 summary가 더 긺.

/var/folders/n3/6fbssxjx1x7bxcnvp_nn7g3r0000gn/T/ipykernel_25744/1068426509.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


{'history': [SystemMessage(content='Boyon introduces themselves and mentions they live in Paris. The AI expresses admiration for this fact. Boyon agrees, stating that Paris is very beautiful.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I wish I could go', additional_kwargs={}, response_metadata={})]}

### 5.4 ConversationKGMemory
* <b>Conversational Knowledge Graph Memory</b>: 대화 중인 entity의 knowledge graph를 만듦.


In [86]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationKGMemory

llm = ChatOpenAI(temperature=0.1)
memory=ConversationKGMemory(llm=llm, return_messages=True)

def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})


add_message("Hi! I'm Boyon, I live in Paris.", "Wow that is so cool!")
# print(memory.load_memory_variables({"input":"Who is Boyoon?"})) #entity의 knowledge graph를 만들어 요약본을 생성, 메모리에 저장
add_message("I love Kimchi.", "That's cool.")
print(memory.load_memory_variables({"input":"Who is Boyoon?"}))

{'history': [SystemMessage(content='On Boyon: Boyon lives in Paris.', additional_kwargs={}, response_metadata={})]}


### 5.5 Memory on LLMChain
* LLM chain: off-the-shelf chain. 일반적인 목적. 빠르게 시작할 수 있음.

In [92]:
# LLMChain에 우리가 가진 메모리 넣기
from langchain.chains import LLMChain

from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

llm=ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    memory_key = "chat_history" #template안에 넣을 history의 위치를 알려줌
)

template = """
    {chat_history}
    Human: {question}
    You: 
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True, #chain의 프롬프트 로그들을 확인할 수 있음
)

print(chain.predict(question="My name is Boyoon."))
print(chain.predict(question="I live in Paris since 7 years ago, but I was born and raised in Seoul."))
print(chain.predict(question="While I was traveling, a stranger asked me where I'm from. How do I answer to that?"))



> Entering new LLMChain chain...
Prompt after formatting:

    
    Human: My name is Boyoon.
    You: 


> Finished chain.
Nice to meet you, Boyoon! How can I assist you today?


> Entering new LLMChain chain...
Prompt after formatting:

    Human: My name is Boyoon.
AI: Nice to meet you, Boyoon! How can I assist you today?
    Human: I live in Paris since 7 years ago, but I was born and raised in Seoul.
    You: 


> Finished chain.
That's interesting! How do you like living in Paris compared to Seoul?


> Entering new LLMChain chain...
Prompt after formatting:

    System: The human introduces themselves as Boyoon. The AI responds by saying "Nice to meet you, Boyoon! How can I assist you today?"
Human: I live in Paris since 7 years ago, but I was born and raised in Seoul.
AI: That's interesting! How do you like living in Paris compared to Seoul?
    Human: While I was traveling, a stranger asked me where I'm from. How do I answer to that?
    You: 


> Finished chain.
You can simp

### 5.6 Chat Based Memory
* memory.load_memory_variables({})가 문자열이 아니라 메시지 객체를 반환하게 합시다

In [ ]:
# LLMChain에 우리가 가진 메모리 넣기
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

llm=ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    memory_key = "chat_history",
    return_messages=True
)

prompt=ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human."),
    MessagesPlaceholder(variable_name="chat_history"), #누가 보냈는지 알 수 없고 예측하기 어려운 양의 메시지를 넣을 수 있음
    ("human":{question}),
])

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

print(chain.predict(question="My name is Boyoon."))
print(chain.predict(question="I live in Paris since 7 years ago, but I was born and raised in Seoul."))
print(chain.predict(question="What's my name?"))

### 5.7 LCEL Based Memory

In [104]:
from langchain.schema.runnable import RunnablePassthrough

from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm=ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    memory_key = "history",
    return_messages=True
)

prompt=ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human."),
    MessagesPlaceholder(variable_name="history"), #누가 보냈는지 알 수 없고 예측하기 어려운 양의 메시지를 넣을 수 있음
    ("human","{question}")
])

def load_memory(_):
    print(_)
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm 

def invoke_chain(question):
    result = chain.invoke({"question":question})
    memory.save_context(
        {"input":question}, 
        {"output":result.content}
    )
    print(result)

invoke_chain("My name is Boyoon")
invoke_chain("What is my name?")

input_variables=['history', 'question'] input_types={'history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.messages.func